# 스파크 고급 분석 5장

## K-평균 군집화로 네트워크 이상 탐지하기

In [1]:
%%configure -f
{
    "proxyUser": "hduser",
    "driverMemory": "4000M", 
    "conf": {"spark.jars.packages": "graphframes:graphframes:0.3.0-spark2.0-s_2.11",
             "spark.master": "local[2]",
             "spark.sql.crossJoin.enabled": "true"}
}

In [2]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,None,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}


In [3]:
val base = "hdfs://localhost:54310/kddcup/"


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

base: String = hdfs://localhost:54310/kddcup/


## 학습/평가 데이터 준비

In [4]:
val dataWithoutHeader = spark.read.option("inferSchema", "true").option("header", false).csv(base + "kddcup.data_10_percent")

val colNames = Seq(
    "duration", "protocol_type", "service", "flag",
    "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
    "hot", "num_failed_logins", "logged_in", "num_compromised",
    "root_shell", "su_attempted", "num_root", "num_file_creations",
    "num_shells", "num_access_files", "num_outbound_cmds",
    "is_host_login", "is_guest_login", "count", "srv_count",
    "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
    "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
    "dst_host_count", "dst_host_srv_count",
    "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
    "dst_host_serror_rate", "dst_host_srv_serror_rate",
    "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
    "label")

val data = dataWithoutHeader.toDF(colNames: _*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataWithoutHeader: org.apache.spark.sql.DataFrame = [_c0: int, _c1: string ... 40 more fields]
colNames: Seq[String] = List(duration, protocol_type, service, flag, src_bytes, dst_bytes, land, wrong_fragment, urgent, hot, num_failed_logins, logged_in, num_compromised, root_shell, su_attempted, num_root, num_file_creations, num_shells, num_access_files, num_outbound_cmds, is_host_login, is_guest_login, count, srv_count, serror_rate, srv_serror_rate, rerror_rate, srv_rerror_rate, same_srv_rate, diff_srv_rate, srv_diff_host_rate, dst_host_count, dst_host_srv_count, dst_host_same_srv_rate, dst_host_diff_srv_rate, dst_host_same_src_port_rate, dst_host_srv_diff_host_rate, dst_host_serror_rate, dst_host_srv_serror_rate, dst_host_rerror_rate, dst_host_srv_rerror_rate, label)
data: org.apache.spark.sql.DataFrame = [duration: int, protocol_type: string ... 40 more fields]


In [5]:
data.head

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res3: org.apache.spark.sql.Row = [0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.]


## KMeas 로 군집 형성 예시 (k=2)

In [6]:
val numericOnly = data.drop("protocol_type", "service", "flag").cache()   // 일단 string 타입은 drop

val assembler = new VectorAssembler().
    setInputCols(numericOnly.columns.filter( _ != "label")).
    setOutputCol("featureVector")

val kmeans = new KMeans().
//     setSeed(1L).
//     setK(2).
    setFeaturesCol("featureVector").
    setPredictionCol("cluster") 

val pipeline = new Pipeline().setStages(Array(assembler, kmeans))

val pipelineModel = pipeline.fit(numericOnly)
val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]

kmeansModel.clusterCenters.foreach(println)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

numericOnly: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [duration: int, src_bytes: int ... 37 more fields]
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_8275d28d54b2
kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_0cc7d4bb2def
pipeline: org.apache.spark.ml.Pipeline = pipeline_758873ba8a0b
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_758873ba8a0b
kmeansModel: org.apache.spark.ml.clustering.KMeansModel = kmeans_0cc7d4bb2def
[47.979395571029514,1622.078830816566,868.5341828266062,4.453261001578883E-5,0.006432937937735314,1.4169466823205539E-5,0.03451682118132869,1.5181571596291647E-4,0.14824703453301485,0.01021213716043885,1.1133152503947209E-4,3.6435771831099954E-5,0.011351767134933808,0.0010829521072021374,1.0930731549329986E-4,0.0010080563539937655,0.0,0.0,0.0013865835391279706,332.2862475203433,292.9071434354884,0.17668541759442902,0.17660780940042928,0.05743309987449906,0.05771839196793657,0.7915488441768395,0.02098164041942

### 군집에 들어가는 데이터 살펴보기

In [7]:
val withCluster = pipelineModel.transform(numericOnly)

withCluster.
    groupBy("cluster", "label").count.
    orderBy($"cluster", $"count".desc).show(false)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

withCluster: org.apache.spark.sql.DataFrame = [duration: int, src_bytes: int ... 39 more fields]
+-------+----------------+------+
|cluster|label           |count |
+-------+----------------+------+
|0      |smurf.          |280790|
|0      |neptune.        |107201|
|0      |normal.         |97278 |
|0      |back.           |2203  |
|0      |satan.          |1589  |
|0      |ipsweep.        |1247  |
|0      |portsweep.      |1039  |
|0      |warezclient.    |1020  |
|0      |teardrop.       |979   |
|0      |pod.            |264   |
|0      |nmap.           |231   |
|0      |guess_passwd.   |53    |
|0      |buffer_overflow.|30    |
|0      |land.           |21    |
|0      |warezmaster.    |20    |
|0      |imap.           |12    |
|0      |rootkit.        |10    |
|0      |loadmodule.     |9     |
|0      |ftp_write.      |8     |
|0      |multihop.       |7     |
+-------+----------------+------+
only showing top 20 rows



## K 선정하기

In [8]:
def clusterScore(df:DataFrame, k:Int) = {
    val assembler = new VectorAssembler().
        setInputCols(numericOnly.columns.filter( _ != "label")).
        setOutputCol("featureVector")

    val kmeans = new KMeans().
        setSeed(1L).
        setK(k).
        setMaxIter(40).  // default : 20
        setTol(1.0e-5). 
        setFeaturesCol("featureVector").
        setPredictionCol("cluster") 

    val pipeline = new Pipeline().setStages(Array(assembler, kmeans))

    val pipelineModel = pipeline.fit(df)
    val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
    
    kmeansModel.summary.trainingCost
    //kmeansModel.computeCost(assembler.transform(df)) / df.count
}

(20 to 100 by 20).map( k => (k, clusterScore(numericOnly, k))).foreach(println)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

clusterScore: (df: org.apache.spark.sql.DataFrame, k: Int)Double
(20,3.4534470232165543E13)
(40,1.6863401552513182E13)
(60,1.592796307162011E13)
(80,1.3780873599313395E13)
(100,1.2477951244097455E13)


### 특징 정규화 후 최적  k 다시 찾기

In [45]:
def clusterScoreWithNormalized(df:DataFrame, k:Int) = {
    val assembler = new VectorAssembler().
        setInputCols(numericOnly.columns.filter( _ != "label")).
        setOutputCol("featureVector")
    
    val scaler = new StandardScaler()
      .setInputCol("featureVector")
      .setOutputCol("scaledFeatureVector")
      .setWithStd(true)
      .setWithMean(false)    // KMEANS는 translation 해도 결과에 무관
     
    val kmeans = new KMeans().
        setSeed(1L).
        setK(k).
        setMaxIter(40).  // default : 20
        setTol(1.0e-5). 
        setFeaturesCol("scaledFeatureVector").
        setPredictionCol("cluster") 

    val pipeline = new Pipeline().setStages(Array(assembler, scaler, kmeans))
    val pipelineModel = pipeline.fit(df)
    val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
    
    kmeansModel.summary.trainingCost
    //kmeansModel.computeCost(assembler.transform(df)) / df.count
}

(20 to 100 by 20).map( k => (k, clusterScoreWithNormalized(numericOnly, k))).foreach(println)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.StandardScaler
clusterScoreWithNormalized: (df: org.apache.spark.sql.DataFrame, k: Int)Double
(20,3732362.744115213)
(40,1024412.0304890894)
(60,538699.248454132)
(80,385748.6877572641)
(100,269503.7740490003)


## 범주형 변수 살리기  (StringIndexer, OneHotEncoder)

In [53]:
def oneHotPipeline(inputCol:String): (Pipeline, String) = {
    
    val indexer = new StringIndexer().
        setInputCol(inputCol).
        setOutputCol(inputCol + "_indexed")
    
    val encoder = new OneHotEncoder().
        setInputCol(inputCol + "_indexed").
        setOutputCol(inputCol + "_vec")
    
    val pipeline = new Pipeline().setStages(Array(indexer, encoder))
    
    (pipeline, inputCol + "_vec")   
}

def clusterScoreWithOneHot(df:DataFrame, k:Int) = {
    
    val (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline("protocol_type")
    val (serviceEncoder, serviceVecCol) = oneHotPipeline("service")
    val (flagEncoder, flagVecCol) = oneHotPipeline("flag")
    
    val inputCols = numericOnly.columns.filter( _ != "label") ++ Seq(protoTypeVecCol, serviceVecCol, flagVecCol)
    val assembler = new VectorAssembler().
        setInputCols(inputCols).
        setOutputCol("featureVector")
    
    val scaler = new StandardScaler()
      .setInputCol("featureVector")
      .setOutputCol("scaledFeatureVector")
      .setWithStd(true)
      .setWithMean(false)    // KMEANS는 translation 해도 결과에 무관
     
    val kmeans = new KMeans().
        setSeed(1L).
        setK(k).
        setMaxIter(40).  // default : 20
        setTol(1.0e-5). 
        setFeaturesCol("scaledFeatureVector").
        setPredictionCol("cluster") 

    val pipeline = new Pipeline().setStages(Array(protoTypeEncoder, serviceEncoder, flagEncoder, assembler, scaler, kmeans))
    val pipelineModel = pipeline.fit(df)
    val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
    
    kmeansModel.summary.trainingCost
    //kmeansModel.computeCost(assembler.transform(df)) / df.count
}


data.cache()

(200 to 260 by 20).map( k => (k, clusterScoreWithOneHot(data, k))).foreach(println)

data.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}
oneHotPipeline: (inputCol: String)(org.apache.spark.ml.Pipeline, String)
clusterScoreWithOneHot: (df: org.apache.spark.sql.DataFrame, k: Int)Double
res147: data.type = [duration: int, protocol_type: string ... 40 more fields]
(200,644140.150984261)
(220,546771.4262406854)
(240,468787.3598376977)
(260,420861.01543326344)
res151: data.type = [duration: int, protocol_type: string ... 40 more fields]


## Anomaly detection 

In [83]:
def getBestModel(df:DataFrame, k:Int) = {
    val (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline("protocol_type")
    val (serviceEncoder, serviceVecCol) = oneHotPipeline("service")
    val (flagEncoder, flagVecCol) = oneHotPipeline("flag")

    val inputCols = numericOnly.columns.filter( _ != "label") ++ Seq(protoTypeVecCol, serviceVecCol, flagVecCol)
    val assembler = new VectorAssembler().
        setInputCols(inputCols).
        setOutputCol("featureVector")

    val scaler = new StandardScaler()
      .setInputCol("featureVector")
      .setOutputCol("scaledFeatureVector")
      .setWithStd(true)
      .setWithMean(false)    // KMEANS는 translation 해도 결과에 무관

    val kmeans = new KMeans().
        setSeed(1L).
        setK(k).
        setMaxIter(40).  // default : 20
        setTol(1.0e-5). 
        setFeaturesCol("scaledFeatureVector").
        setPredictionCol("cluster") 

    val pipeline = new Pipeline().setStages(Array(protoTypeEncoder, serviceEncoder, flagEncoder, assembler, scaler, kmeans))
    pipeline.fit(df)
}

data.cache()

val pipeline = getBestModel(data, 260)

val kMeansModel = pipeline.stages.last.asInstanceOf[KMeansModel]
val centroids = kMeansModel.clusterCenters
val predictions = pipeline.transform(data)

// centroid사의의 거리를 구하고 100 번째 거리를 threshold로 정한다.
val threshold = predictions.select("cluster", "scaledFeatureVector").as[(Int, Vector)].
    map { case (cluster, vec) => 
        Vectors.sqdist(centroids(cluster), vec)
    }.
    orderBy($"value".desc).take(100).last

data.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

getBestModel: (df: org.apache.spark.sql.DataFrame, k: Int)org.apache.spark.ml.PipelineModel
res225: data.type = [duration: int, protocol_type: string ... 40 more fields]
pipeline: org.apache.spark.ml.PipelineModel = pipeline_39293a8df720
kMeansModel: org.apache.spark.ml.clustering.KMeansModel = kmeans_3bfe15c0fed5
centroids: Array[org.apache.spark.ml.linalg.Vector] = Array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10060435343008005,0.02637193415721355,2.617728048009747,2.6237795820928587,0.014620906010109434,0.0012770582696820874,0.9136059317621815,2.7458512969965354,0.004324061900025904,3.9384344879990563,0.08516361323944795,0.08607793681410496,0.7409707902151725,0.010423867296165644,0.0,2.6093223610926355,2.625226947857916,0.006329520858093144,0.0,0.0,2.0553630063998165,0.0,2.3147536376709317,0.1016688615847554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### anomaliy 검출

In [89]:
val anomalies = predictions.filter { row =>
    val vec = row.getAs[Vector]("scaledFeatureVector")
    val cluster = row.getAs[Int]("cluster")
    
    val dist = Vectors.sqdist(centroids(cluster), vec)
    
    dist >= threshold
}

anomalies.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

anomalies: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [duration: int, protocol_type: string ... 49 more fields]
res262: Long = 100


In [90]:
anomalies.first

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res263: org.apache.spark.sql.Row = [257,tcp,telnet,SF,181,1222,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,62,15,0.21,0.05,0.02,0.13,0.03,0.13,0.0,0.0,normal.,1.0,(2,[1],[1.0]),11.0,(65,[11],[1.0]),0.0,(10,[0],[1.0]),(115,[0,1,2,8,13,19,20,25,28,29,30,31,32,33,34,35,39,51,105],[257.0,181.0,1222.0,1.0,2.0,1.0,1.0,1.0,62.0,15.0,0.21,0.05,0.02,0.13,0.03,0.13,1.0,1.0,1.0]),(115,[0,1,2,8,13,19,20,25,28,29,30,31,32,33,34,35,39,51,105],[0.3631243814792357,1.83157948440338E-4,0.03698547075334117,2.814168444874875,20.74347117165251,0.0046915887460480836,0.004059713227290898,2.5760614800997756,0.9575972778839426,0.14145547123294683,0.5112213357966311,0.45762773452021277,0.04155332553971481,3.0854766450748023,0.0788243407356959,0.3412795032215277,2.055363006399789,31.0483974914...